In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime

In [5]:
# This function calculates volatility for a given symbol for the date
# a hurricane hit and caculates the average volatility for that same day
# for every year a hurricane did not hit. It returns this data
# aligned in a pandas dataframe

def same_day_algo(yahoo_ticker):
    
    # this first section of the function calculates volatility
    # however this will only be for years hurricanes did not hit
    # I will calculate volatility for years hurricane hits at the end of the function
    def same_day_calc(yahoo_ticker):
        name = yahoo_ticker
        x = yf.Ticker(yahoo_ticker).history(period='MAX')
        high = x['High']
        low = x['Low']
        mean = (abs(high)+abs(low))/2
        dr = abs(high-low)
        ans = ((dr/mean)*1000)+10
        hi = pd.DataFrame(data=x)
        hi['Wills Volatility Measure'] = ans
        hi['Date'] = hi.index
        
        # I needed to export this to a CSV and read back in for it to work
        # as with the previous piece of software
        hi.to_csv('in progress.csv') 
        data = pd.read_csv('in progress.csv')
        
        
        # this aligns next section aligns hurricane dates with
        # volatility and every date of the matching day of the year
        # a hurricane did not hit
        def Align_days(data, name):
            hurricanes = pd.read_csv('hurricanes_scraped.csv') # read in hurricane data
            hurricanes = hurricanes.rename(columns={'Date of closest approach': 'Date'})
            for x in range(0,len(hurricanes)): # this loop formats date column to stock dates
                hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")
            
            # create empty list to append formatted dates
            datem = []
            # loop through data and formats dates as required and appends to list
            for x in range(0,len(hurricanes)):
                datem.append(datetime.datetime.strptime(hurricanes['Date'][x], "%Y-%m-%d"))

            # there is going to be lists of lists this time so I will use a dictionary
            # I created an empty dictionary to fill with my data
            dict = {}
            
            # loops through dataframe produced and read back in 
            # in first part of the function and formats to match hurricane dates format
            for y in range(0,len(data)):
                data['Date'][y] = datetime.datetime.strptime(data['Date'][y], "%Y-%m-%d")
            
            # Loop through datem list with the hurricane dates and use
            # three booleans to ensure day and month but not year match
            for i in range(0, len(datem)):
    
                # create empty list to append matched dates
                hello_moto = []
                
                # loops through data and extracts volatility measures for years
                # hurricane didn't hit and appends to list
                for j in range(0, len(data)):
                    if datem[i].month == data['Date'][j].month and datem[i].day == data['Date'][j].day and datem[i].year != data['Date'][j].year:
                        hello_moto.append(data['Wills Volatility Measure'][j])
                        
                        # calculate the average value of volatility for years
                        # hurricane didn't hit
                        bug = sum(hello_moto)/len(hello_moto)
                        
                        # add this result to dictionary aligned to date hurricane did hit
                        dict[i] = bug, datem[i].month, datem[i].day, datem[i].year
            
            # export dictionary created to Pandas dataframe
            df = pd.DataFrame.from_dict(dict, orient = 'index')
            
            # I need to rename the columns
            df = df.rename(columns={0: "Mean volatility on this day", 1: "Month", 2: "Day", 3:"Year"})
            
            # I need to format data to have leading zeros
            # Convert to string
            df['Month'] = df['Month'].apply(str)
            # Adds leading zeros
            df['Month'] = df['Month'].apply(lambda x: x.zfill(2))
            df['Day'] = df['Day'].apply(str)
            df['Day'] = df['Day'].apply(lambda x: x.zfill(2))
            df['Date'] = df['Month']+'-'+df['Day']
            df['Year'] = df['Year'].apply(str)
            # adds correctly formatted date
            df['Actual Date'] = df['Year']+'-'+df['Date']
            # we do not need these two columns now so I will delete them
            del df['Day']
            del df['Month']
            
            # I don't want to alter my hurricanes data,
            # I will need it in the next section of the function
            # I define a new dataframe as hurricane2 to work with
            hurricanes_2 = hurricanes

            # I need to format dates again
            for j in range(0,len(hurricanes_2)):
                hurricanes_2["Date"][j] = hurricanes_2["Date"][j][5:]
            
            # I can return dataframe
            return(df)
        
        # I can return everything so far to the outter function
        return(Align_days(data,name))
    
    # I can now calculate volatility once again for the dates hurricanes hit
    same_day_calc(yahoo_ticker).to_csv('df_in_progress.csv')
    name = yahoo_ticker
    x = yf.Ticker(yahoo_ticker).history(period='MAX')
    high = x['High']
    low = x['Low']
    mean = (abs(high)+abs(low))/2
    dr = abs(high-low)
    ans = ((dr/mean)*1000)+10
    hi = pd.DataFrame(data=x)
    hi['Wills Volatility Measure'] = ans
    hi['Date'] = hi.index

    hi.to_csv('in progress.csv') # I needed to export this to a CSV and read back in for it to work
    data = pd.read_csv('in progress.csv')
    
    # this takes in data from csv just created
    def Align_my_data(data, name):
        hurricanes = pd.read_csv('df_in_progress.csv') # read in hurricane data

        # create empty lists to append data to
        zing = data
        zing_box = []
        ticker = []
        date_box = []
        hurricane_box = []
        that_day_box = []

        # loops through hurricane and volatility data
        # appends matched dates and volatility so all data is aligned
        for x in range(0,len(hurricanes['Actual Date'])): # loops over hurricane hit dates
            for y in range(0, len(zing['Date'])): # loops over dates of time series to align with hurricanes
                if hurricanes['Actual Date'][x] == zing['Date'][y]: # uses boolean to check whether dates match
                    zing_box.append(zing['Wills Volatility Measure'][y])
                    date_box.append(hurricanes['Actual Date'][x])
                    that_day_box.append(hurricanes['Mean volatility on this day'][x])
            
                    ticker.append(name)
                
        # combine these lists into a Pandas dataframe
        result = pd.DataFrame({'Symbol': ticker, 'Hurricane Date': date_box,'Wills Volatility Measure': zing_box, 'Mean volatility on this day other years': that_day_box})
        
        # cleans data before returning result
        # as before, volatility is calculated to many decimal places
        # any zero values will therefore be taken as incorrect and removed
        for x in range(0,len(result)): # now we need to drop zero values
            if result['Wills Volatility Measure'][x] == 10:
                result.drop(index = [x] , inplace = True)
        
        # can now return this to the outer outer function
        return(result)
    
    # can now return the resulting dataframe
    return(Align_my_data(data,name))

In [8]:
sandp = same_day_algo("^GSPC")
sandp # Great, this works!

<ipython-input-5-6f7168519bdd>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")
<ipython-input-5-6f7168519bdd>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'][y] = datetime.datetime.strptime(data['Date'][y], "%Y-%m-%d")
<ipython-input-5-6f7168519bdd>:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes_2["Date"][j] = hurrican

,Symbol,Hurricane Date,Wills Volatility Measure,Mean volatility on this day other years
0,^GSPC,2003-09-18,23.950302,22.404070
1,^GSPC,2004-09-16,15.065872,21.803806
2,^GSPC,2004-08-13,16.446446,21.121322
3,^GSPC,2004-09-16,15.065872,21.803806
4,^GSPC,2005-08-29,20.555466,20.214581
5,^GSPC,2005-08-25,13.433363,21.378588
6,^GSPC,2005-10-24,26.645831,23.252586
7,^GSPC,2005-08-29,20.555466,20.214581
8,^GSPC,2005-08-29,20.555466,20.214581
9,^GSPC,2016-10-07,19.747818,23.353128


In [ ]:
# The runtime is not good. Therefore, I will once again
# use the function on indices one at a time for ease of use.
# I will now use it to get datafor US indices

In [4]:
nasdaq = same_day_algo("^IXIC")

<ipython-input-3-acfe65e93c67>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")
<ipython-input-3-acfe65e93c67>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'][y] = datetime.datetime.strptime(data['Date'][y], "%Y-%m-%d")
<ipython-input-3-acfe65e93c67>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes_2["Date"][j] = hurricane

In [5]:
nasdaq.to_csv('same day nasdaq.csv')

In [6]:
dji = same_day_algo("^DJI")

<ipython-input-3-acfe65e93c67>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")
<ipython-input-3-acfe65e93c67>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'][y] = datetime.datetime.strptime(data['Date'][y], "%Y-%m-%d")
<ipython-input-3-acfe65e93c67>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes_2["Date"][j] = hurricane

In [7]:
rus2000 = same_day_algo("^RUT")

<ipython-input-3-acfe65e93c67>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")
<ipython-input-3-acfe65e93c67>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'][y] = datetime.datetime.strptime(data['Date'][y], "%Y-%m-%d")
<ipython-input-3-acfe65e93c67>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes_2["Date"][j] = hurricane

In [8]:
rus3000 = same_day_algo("^RUA")

<ipython-input-3-acfe65e93c67>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")
<ipython-input-3-acfe65e93c67>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'][y] = datetime.datetime.strptime(data['Date'][y], "%Y-%m-%d")
<ipython-input-3-acfe65e93c67>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes_2["Date"][j] = hurricane

In [9]:
rus1000 = same_day_algo("RUI")

<ipython-input-3-acfe65e93c67>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes['Date'][x] = hurricanes['Date'][x].replace("00:00:00", "").replace(" ", "")
<ipython-input-3-acfe65e93c67>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'][y] = datetime.datetime.strptime(data['Date'][y], "%Y-%m-%d")
<ipython-input-3-acfe65e93c67>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hurricanes_2["Date"][j] = hurricane

In [ ]:
# I will concat this data into one file

In [10]:
US_indicies = pd.concat([sandp,dji,nasdaq,rus2000,rus3000], axis = 0)

In [ ]:
# I need to rename the index ticker symbols to the index name

In [11]:
US_indicies['Symbol'] = US_indicies['Symbol'].replace('^DJI', 'Dow Jones 30')
US_indicies['Symbol'] = US_indicies['Symbol'].replace('^IXIC', 'Nasdaq')
US_indicies['Symbol'] = US_indicies['Symbol'].replace('^RUT', 'Russel 2000')
US_indicies['Symbol'] = US_indicies['Symbol'].replace('^GSPC', 'S&P 500')
US_indicies['Symbol'] = US_indicies['Symbol'].replace('^IXIC', 'Nasdaq')
US_indicies['Symbol'] = US_indicies['Symbol'].replace('^RUA', 'Russel 3000')

In [12]:
US_indicies

,Symbol,Hurricane Date,Wills Volatility Measure,Mean volatility on this day other years
0,S&P 500,2003-09-18,23.950302,22.404070
1,S&P 500,2004-09-16,15.065872,21.803806
2,S&P 500,2004-08-13,16.446446,21.121322
3,S&P 500,2004-09-16,15.065872,21.803806
4,S&P 500,2005-08-29,20.555466,20.214581
...,...,...,...,...
25,Russel 3000,1998-08-27,54.219081,21.969726
26,Russel 3000,1999-09-16,25.930909,21.452847
27,Russel 3000,1999-10-18,25.871310,21.515756
28,Russel 3000,1999-08-23,26.824301,21.755002


In [ ]:
# This is ready to export as a csv!

In [13]:
US_indicies.to_csv('US indicies.csv')